In [87]:
import plotly.express as px
import pandas as pd
import numpy as np
from test import run_simulation

In [89]:
df = run_simulation(
    50, 10, 100
)

initializing habitats
value of habitat: 16.664999999999996
50 habitats created
initializing fish
starting processes
building exchanger
initializing state
running simulation


100%|██████████| 100/100 [00:00<00:00, 118.50it/s]

process finishedprocess finished
process finishedprocess finished
process finishedprocess finishedprocess finishedprocess finishedprocess finished





process finished

sending stop signals
waiting for processes to finish
clearing sub queues
writing results


In [94]:
def watch_simulation(df, color='win'):
    jitter = []
    for tag in df['tag'].unique():
        jitter.append({
            'tag': tag,
            'jitter': np.random.uniform(-1, 1)
        })
    jitter = pd.DataFrame(jitter)
    df = df.merge(jitter, on='tag')

    return px.scatter(
        df, x="position", y="jitter", size="weight", color=color, hover_name="tag",
        range_x=[-0.1,1.1], range_y=[-1.1,1.1],
        animation_frame="time", animation_group="tag"
    )

watch_simulation(df)

In [95]:
def watch_props(df, bin_size = 0.2, agg={'weight': 'sum'}):
    df['bin'] = df['position'] // bin_size
    gdf = df.groupby(['bin', 'time']).agg(agg).reset_index().rename({'weight': 'count'}, axis=1)
    full_index = []
    for bin in gdf['bin'].unique():
        for time in gdf['time'].unique():
            full_index.append({
                'bin': bin,
                'time': time
            })
    full_index = pd.DataFrame(full_index)
    gdf = full_index.merge(gdf, on=['bin', 'time'], how='left').fillna(0)
    cbt = gdf.groupby('time').agg({'count': 'sum'}).reset_index().rename({'count': 'total'}, axis=1)
    gdf = gdf.merge(cbt, on='time')
    gdf['proportion'] = gdf['count'] / gdf['total']
    return px.bar(gdf, x='bin', y='proportion', animation_frame='time', range_y=[0, 0.5])

watch_props(df)